In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title Initial Imports + Helper functions
import tensorflow as tf
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
import time
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K

from tensorflow.keras.utils import to_categorical  ## maintain tf.keras always


def cosine_similarity(a, b):
### https://stackoverflow.com/questions/18424228/cosine-similarity-between-2-number-lists
    cos_sim = np.inner(a, b) / (norm(a) * norm(b))    
    return cos_sim

def get_various_metrics_and_print(Y_true, Y_predicted):
    TN, FP, FN, TP = confusion_matrix(Y_true, Y_predicted).ravel()
    accuracy = (TP + TN)/(TP+TN+FP+FN)
    recall = (TP)/(TP + FN)
    specificity = (TN)/(TN + FP) # TNR
    false_positive_rate = (FP)/(TN + FP) # false_positive_rate = 1 - TNR
    precision = (TP)/(TP + FP)
    false_discovery_rate = (FP)/(TP + FP)
    neg_predicted_val = (TN)/(TN + FN)
    f1_score = 2*((precision * recall) / (precision + recall))

    print("TN = ", TN, " FP = ", FP, " FN = ", FN, " TP = ", TP)
    print("Accuracy = ", accuracy*100, "%")
    print("TPR = Sensitivity = Recall = ", recall*100, "%")
    print("TNR = Specificity = ", specificity*100, "%")
    print("Precision = PPV = Positive Predictive Value = ", precision*100, "%")
    print("FDR = False Discovery Rate = ", false_discovery_rate*100, "%")
    print("FPR = False Positive Rate = ", false_positive_rate*100, "%")
    print("F1 Score = ", f1_score*100, "%")
    print("Neg Predicted Val = ", neg_predicted_val*100, "%")
    print("\n")
    c_report = classification_report(y_true=Y_true, y_pred=Y_predicted)
    print(c_report)
    # return precision, recall, f1_score

def plot_confusion_matrix(Y_true=None, Y_predicted=None):
    ## https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
    cf_matrix = confusion_matrix(Y_true, Y_predicted)
    print(cf_matrix)
    # group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_names = ['TN','FP','FN','TP']
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap=plt.cm.magma)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")


"""
  Loads the stop words into a list.
"""
def load_stop_words_list_and_dict(file):
  fin = open(file, 'r', encoding="utf-8") # open file
  st_words = fin.readlines() # load st-words into variable [list]
  fin.close() # close file
  stop_words_list = []
  for word in st_words: # remove new-lines/other spaces
    word = word.strip()
    stop_words_list.append(word)
  
  stop_words_dict = {} # dictionary of key-value pairs. [to do easy checking]
  for sw in stop_words_list:
          stop_words_dict[sw.strip()] = "" # for each stop word, it's replacement will be an empty-string
          
  return stop_words_list, stop_words_dict


"""
  Given a sentence, it removes the stop-words and other things [stemming]
  Input: sentence, list of stop-words
  Output: stemmed sentence
"""
def obtain_stemmed_sentence(sentence, stop_words_dict): # Base code: Rafi
  stemmed_sentence = sentence # first copy the sentence
  stemmed_sentence = stemmed_sentence.replace("\n", "") # replace new line with empty-string
  # remove english words
  stemmed_sentence = re.sub(r'[।\\/“”0-9@&$%+_=<>~*#০১২৩৪৫৬৭৮৯…(){}\[\]\?\!\.a-zA-Z\…\|]+','',stemmed_sentence) # replace theese with empty-string [Mahim]
  sentences_split = re.split('[–—\s.,\-\‘\’\‘\'\\\":]',stemmed_sentence) # split wrt these delims
  label = ""
  for x in sentences_split:
    if x not in stop_words_dict and len(x) >= 2: # if word is not a stop-word append by adding a space
        label = label + " " + x
#   label = label.replace("  ", " ") # replace double spaces with one space
  return label

  
def read_from_file_and_load_vocab():
    vocab_list = []
    in_file = "/content/drive/My Drive/Data_Initial/Bangla-Vocab-22-Aug-withoutEnglish.txt"
    print(f"in_file = {in_file}")
    fin1 = open(in_file,'r', encoding="utf-8")
    for line in fin1.readlines():
        if len(line) > 1:
            vocab_list.append(line.strip())
    fin1.close()
    return vocab_list


## write_to_file_vocab()


def get_vocab_dictionary(vocab_list):
    vocab_dict = {}
    cnt = 0
    for word in vocab_list:
        if word not in vocab_dict:
            vocab_dict[word] = cnt
        cnt += 1
    return vocab_dict



def get_encoded_content(stemmed_sentence, vocab_dict):
    list_encoded_content = []
    for word in stemmed_sentence.split():
        if word in vocab_dict:
            encoded_int = vocab_dict[word] + 1 # since '0' is used for padding
            list_encoded_content.append(encoded_int)
    return list_encoded_content


def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def get_recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return recall

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
RANDOM_STATE = 2245 # for ensuring reproducible results.
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

LEN_DOC_CONTENT = 400
file_stop_words_bangla = "/content/drive/My Drive/Data_Initial/STOP_WORDS_BANGLA.txt"
FILE_TRAIN_AND_VALIDATION ="/content/drive/My Drive/Data_Initial/contents-labels-train-val.csv"

DATE_TIME = "7Sept-4am"

def get_fileModel_and_fileWeights(num, model_name="CNN", NUM_MODELS = 10, synth_type="No", BASE_PATH="/content/drive/My Drive/Ensembling/Ens-Models/"):
    file_name = BASE_PATH + "ens-"+ model_name + "-numModels-" + str(NUM_MODELS) + "-" + synth_type + "Synth-" + str(DATE_TIME) + "-" + str(num) + ".h5"
    file_weights = "IGNORE"
    return file_name, file_weights

fm, fw = get_fileModel_and_fileWeights(num=0, model_name="biLSTM", synth_type="Word")
print(fm)

stop_words_list, stop_words_dict = load_stop_words_list_and_dict(file_stop_words_bangla)
vocab_list = read_from_file_and_load_vocab()
print("Size of list of vocab", len(vocab_list))
vocab_dict = get_vocab_dictionary(vocab_list)
print(len(vocab_dict))
vocab_size = len(vocab_dict) + 1
print(f"vocab_size = {vocab_size}")

/content/drive/My Drive/Ensembling/Ens-Models/ens-biLSTM-numModels-10-WordSynth-7Sept-4am-0.h5
in_file = /content/drive/My Drive/Data_Initial/Bangla-Vocab-22-Aug-withoutEnglish.txt
Size of list of vocab 237931
237931
vocab_size = 237932


In [4]:
FILE_TEST_DATA = "/content/drive/My Drive/Data_Initial/contents-labels-test.csv"
df_test = pd.read_csv(FILE_TEST_DATA)
df_test = df_test.sample(frac=1, random_state=RANDOM_STATE)
print(f"len(df_test) = {len(df_test)}")

len(df_test) = 5033


In [5]:
df_test["stemmed_content"] = df_test.apply(lambda row : obtain_stemmed_sentence(row['content'], stop_words_dict), axis=1)
df_test["encoded_contents"] = df_test.apply(lambda row : get_encoded_content(row['stemmed_content'], vocab_dict=vocab_dict), axis=1)

In [6]:
#@title Required functions for majority voting and df models
## Obtain the series thing.
def obtain_metrics_as_list(clf_model_name, Y_true, Y_predicted):
    # TN, FP, FN, TP = confusion_matrix(Y_true, Y_predicted).ravel()
    
    cm = confusion_matrix(Y_true, Y_predicted)
    TN, FP, FN, TP = cm.ravel()

    TNR = (TN)/(TN + FP) # TNR = Specificity
    FPR = (FP)/(TN + FP) # FPR = False Pos Rate = 1 - TNR
    recall = (TP)/(TP + FN) # TPR = Recall = Sensitivity
    FNR = (FN)/(TP + FN) # False Neg Rate = 1 - TPR
    precision = (TP)/(TP + FP) # FDR = 1 - precision
    neg_predicted_val = (TN)/(TN + FN)  # https://stats.stackexchange.com/questions/424737/is-there-a-name-for-this-metric-tn-tn-fn
    f1_score = 2*((precision * recall) / (precision + recall)) # F1-Score
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    clf_report = classification_report(y_true=Y_true, y_pred=Y_predicted)

    list_return = [] ## form list and append values
    list_return.append(clf_model_name)
    list_return.append(cm)
    list_return.append([TN, FP, FN, TP])
    list_return.append(TNR*100)
    list_return.append(FPR*100)
    list_return.append(recall*100) # recall is same as TPR
    list_return.append(FNR*100)
    list_return.append(f1_score*100)
    list_return.append(precision*100)
    list_return.append(neg_predicted_val*100)
    list_return.append(accuracy*100)
    list_return.append(clf_report)

    return list_return


############# Test the method ################
# metrics = obtain_metrics_as_list(clf_model_name="Keras-Embedding-Model", Y_true=y_test, Y_predicted=y_preds)
# print(metrics)

## Obtain and return dataframe.
def appendAndReturn_DataframeRes(df_models, clf_model_name, Y_true, Y_predicted):
    metrics = obtain_metrics_as_list(clf_model_name=clf_model_name, Y_true=Y_true, Y_predicted=Y_predicted)
    a_series = pd.Series(metrics, index=df_models.columns)
    # print(f"Inside appendAndReturn_DataframeRes() name = {clf_model_name} ... returned series.len = {len(a_series)}, len(metrics) = {len(metrics)}")
    df_models = df_models.append(a_series, ignore_index=True)
    return df_models


def predict_one_model(model_num, encoded_content_this_model, BASE_PATH_ENS_MODELS=None, load_best_weights=False,
        NUM_MODELS_ENS=None, model_name="CNN", synth_type="No"):
    padded_docs_this_model = pad_sequences(encoded_content_this_model, maxlen=LEN_DOC_CONTENT, padding='pre') # pad the sequences.
    
    if BASE_PATH_ENS_MODELS == None:
        file_model_this, file_weights = get_fileModel_and_fileWeights(model_num)
    else:
        file_model_this, file_weights = get_fileModel_and_fileWeights(num=model_num, model_name=model_name,
                NUM_MODELS=NUM_MODELS_ENS, synth_type=synth_type, BASE_PATH=BASE_PATH_ENS_MODELS)

    dependencies = {
        'get_recall': get_recall,
        'get_f1': get_f1
    }
    emb_model_content = tf.keras.models.load_model(file_model_this, custom_objects=dependencies)
    print(f"Loading model from file {file_model_this} done !!")

    if load_best_weights == True:
        print(f"Loading weights from file weights = {file_weights}")
        emb_model_content.load_weights(file_weights)

    ## predict using the model.
    preds_y = emb_model_content.predict(padded_docs_this_model)
    
    ## delete the model.
    del emb_model_content
    del padded_docs_this_model

    # return predictions.
    return np.array(preds_y)

### Count Majority ###
def count_majority(preds_y_all, NUM_MAJORITY_TIE, THRESH, NUM_EXAMPLES=None): # 10 out of 28 if 0 -> then give 0
    y_predicted_ensembled = []
    count_zeros_all_models_per_example = []
    count_ones_all_models_per_example = []
    
    for prediction_example_itr in range(len(preds_y_all[0])):  ## preds_y_all[0] is the predicted-samples of model #0 i.e. num of X_test
        count_zero_this_example = 0
        count_one_this_example = 0
        for model_iter in range(len(preds_y_all)): ## len(preds_y_all) = NUMBER OF MODELS FOR ENSEMBLING.
            prediction_this_model_this_example = preds_y_all[model_iter][prediction_example_itr]
            # print(f"prediction_example_itr = {prediction_example_itr}, prediction_this_model_this_example = {prediction_this_model_this_example}, prediction_this_model_this_example.shape = {prediction_this_model_this_example.shape}, prediction_this_model_this_example[0] = {prediction_this_model_this_example[0]}")
            if prediction_this_model_this_example[0] >= THRESH:
                count_zero_this_example += 1
            else:
                count_one_this_example += 1
        
        ## Append zero/one counts for this example
        count_zeros_all_models_per_example.append(count_zero_this_example)
        count_ones_all_models_per_example.append(count_one_this_example)

    for pred_itr in range(len(preds_y_all[0])):
        if NUM_MAJORITY_TIE == -1:
            ## Normal majority comparison.
            if count_zeros_all_models_per_example[pred_itr] >= count_ones_all_models_per_example[pred_itr]:
                label_ensembled = 0
            else:
                label_ensembled = 1
            y_predicted_ensembled.append(label_ensembled)
        else: ## Use MAJORITY-TIE
            if count_zeros_all_models_per_example[pred_itr] >= NUM_MAJORITY_TIE:
                label_ensembled = 0
            else:
                label_ensembled = 1
            y_predicted_ensembled.append(label_ensembled)

    np_zeros = np.array(count_zeros_all_models_per_example)
    np_ones = np.array(count_ones_all_models_per_example)
    np_pred_ens = np.array(y_predicted_ensembled)

    del y_predicted_ensembled
    del count_ones_all_models_per_example
    del count_zeros_all_models_per_example
    
    return np_zeros, np_ones, np_pred_ens

def obtain_normal_all_models_prediction(X_test, BASE_PATH_ENS_MODELS=None, NUM_MODELS_ENS=None, model_name="CNN", synth_type="No", load_best_weights=False):
    preds_y_all = []
    for model_iter in range(0, NUM_MODELS_ENS):
        preds_y = predict_one_model(model_num=model_iter, encoded_content_this_model=X_test, 
            BASE_PATH_ENS_MODELS=BASE_PATH_ENS_MODELS, load_best_weights=load_best_weights, 
            NUM_MODELS_ENS=NUM_MODELS_ENS, model_name=model_name, synth_type=synth_type)
        
        preds_y_all.append(preds_y)
        print(f"Predicting for Model_iter = {model_iter}")
    return preds_y_all


## Create Ensembled Prediction ##
def create_ensembled_prediction(X_test, THRESH=0.5, NUM_MAJORITY_VOTING=2, BASE_PATH_ENS_MODELS=None, NUM_MODELS_ENS=None):
    ############################### Predict for each model. #########################################
    preds_y_all = []
    for model_iter in range(0, NUM_MODELS_ENS):
        preds_y = predict_one_model(model_num=model_iter, encoded_content_this_model=X_test, BASE_PATH_ENS_MODELS=BASE_PATH_ENS_MODELS)
        preds_y_all.append(preds_y)
        print(f"Predicting for Model_iter = {model_iter}")
        # print(f"Model iter = {model_iter}, unique = {np.unique(preds_y, return_counts=True)}")
    
    ###### Count Majority ########
    (count_zeros, count_ones, y_pred_ens) = count_majority(preds_y_all, NUM_MAJORITY_TIE=NUM_MAJORITY_VOTING, THRESH=THRESH, NUM_EXAMPLES=len(X_test))
    return (count_zeros, count_ones, y_pred_ens)

In [7]:
encoded_contents = df_test['encoded_contents'].values
padded_contents = pad_sequences(sequences=encoded_contents, maxlen=LEN_DOC_CONTENT, padding='pre')

X_test = padded_contents
Y_test = df_test['label'].values

In [8]:
## https://www.kite.com/python/answers/how-to-create-an-empty-dataframe-with-column-names-in-python
column_names =["Model Name", "Confusion Matrix", "[TN, FP, FN, TP]", "TNR/Specificity(%)", 
        "FPR(%)", "TPR/Recall(%)", "FNR(%)",
        "F1-Score(%)", "Precision(%)", "Negative Pred. Value (%)", "Accuracy(%)",
        "CLF-Report-SKL"]

df_models = pd.DataFrame(columns = column_names)
print(df_models.head())
display(df_models.head())

Empty DataFrame
Columns: [Model Name, Confusion Matrix, [TN, FP, FN, TP], TNR/Specificity(%), FPR(%), TPR/Recall(%), FNR(%), F1-Score(%), Precision(%), Negative Pred. Value (%), Accuracy(%), CLF-Report-SKL]
Index: []


,Model Name,Confusion Matrix,"[TN, FP, FN, TP]",TNR/Specificity(%),FPR(%),TPR/Recall(%),FNR(%),F1-Score(%),Precision(%),Negative Pred. Value (%),Accuracy(%),CLF-Report-SKL


In [9]:
BASE_PATH_ENS_MODELS = "/content/drive/My Drive/Ensembling/Ens-Models/"

In [10]:
MODEL_NAME = "LSTM"
print(DATE_TIME)
date_times = [DATE_TIME, DATE_TIME] ## ["2Sept-4am", "2Sept-4am"]
thresholds = [0.1, 0.35, 0.5]
# num_majority_votes_arr = [[9, 8, 7, 6, -1], [3, 2, -1]]

num_majority_votes_arr = [[10, 9, 8, 7 -1], [5, 4, 3, -1]]

num_ensemble_models = [10, 5] # [10, 5]

7Sept-4am


In [11]:
#@title Predictions all for One type of synthesis

print("All are commented out !")

# for itr in range(len(num_ensemble_models)): 
#     itr_ensemble = itr
#     NUM_ENS = num_ensemble_models[itr_ensemble]
#     BASE_PATH_EMBEDDING_MODELS = BASE_PATH_ENS_MODELS ## base_folder_paths[itr_ensemble]
#     DATE_TIME = date_times[itr_ensemble]

#     num_majority_votes = num_majority_votes_arr[itr_ensemble]

#     ### OBTAIN FOR ONE ENSEMBLING_MODEL
#     y_preds_all = obtain_normal_all_models_prediction(X_test=X_test, BASE_PATH_ENS_MODELS=BASE_PATH_ENS_MODELS, 
#             NUM_MODELS_ENS=NUM_ENS, model_name='CNN', synth_type='No', load_best_weights=False)


#     ## Iterate for thresholds array.
#     for THRESHOLD in thresholds:
#         print(f"NUM_ENS = {NUM_ENS}, DATE_TIME = {DATE_TIME}, THRESHOLD = {THRESHOLD}, num_majority_votes = {num_majority_votes}\
# \nBASE_PATH_EMBEDDING_MODELS = {BASE_PATH_EMBEDDING_MODELS}")

#         for NUM_MAJORITY_VOTE in num_majority_votes:
#             print(f"NUM_ENS = {NUM_ENS}, THRESHOLD = {THRESHOLD}, NUM_MAJORITY_VOTE = {NUM_MAJORITY_VOTE}")
            
#             (count_zeros, count_ones, y_pred_ens) = count_majority(y_preds_all, NUM_MAJORITY_TIE=NUM_MAJORITY_VOTE, THRESH=THRESHOLD, NUM_EXAMPLES=len(X_test))
#             print("len y_pred_ens = ", len(y_pred_ens))
#             print(y_pred_ens[0:10])

#             print("Unique y_pred_ens: ", np.unique(y_pred_ens, return_counts=True))

#             model_name_to_save = "ENS_CNN_" + str(NUM_ENS) + "_models_TH_" + str(THRESHOLD) + "_Major_" + str(NUM_MAJORITY_VOTE)
#             print(f"Model_name_to_save = {model_name_to_save}")
            

#             # ## Append to dataframe and csv file.
#             df_models = appendAndReturn_DataframeRes(df_models=df_models, clf_model_name=model_name_to_save,
#                                             Y_true=df_test["label"].values, Y_predicted=y_pred_ens)
#             # ## Delete lists.
#             del count_zeros
#             del count_ones
#             del y_pred_ens
    
#     del y_preds_all


All are commented out !


In [12]:
%%time
for itr in range(len(num_ensemble_models)): 
    itr_ensemble = itr
    NUM_ENS = num_ensemble_models[itr_ensemble]
    BASE_PATH_EMBEDDING_MODELS = BASE_PATH_ENS_MODELS ## base_folder_paths[itr_ensemble]
    DATE_TIME = date_times[itr_ensemble]

    num_majority_votes = num_majority_votes_arr[itr_ensemble]

    for synth_type in ["No", "Word", "Char", "SW"]:
        ### OBTAIN FOR ONE ENSEMBLING_MODEL
        y_preds_all = obtain_normal_all_models_prediction(X_test=X_test, BASE_PATH_ENS_MODELS=BASE_PATH_ENS_MODELS, 
                NUM_MODELS_ENS=NUM_ENS, model_name=MODEL_NAME, synth_type=synth_type, load_best_weights=False)


        ## Iterate for thresholds array.
        for THRESHOLD in thresholds:
            print(f"NUM_ENS = {NUM_ENS}, DATE_TIME = {DATE_TIME}, THRESHOLD = {THRESHOLD}, num_majority_votes = {num_majority_votes}\
    \nBASE_PATH_EMBEDDING_MODELS = {BASE_PATH_EMBEDDING_MODELS}")

            for NUM_MAJORITY_VOTE in num_majority_votes:
                print(f"NUM_ENS = {NUM_ENS}, THRESHOLD = {THRESHOLD}, NUM_MAJORITY_VOTE = {NUM_MAJORITY_VOTE}")
                
                (count_zeros, count_ones, y_pred_ens) = count_majority(y_preds_all, NUM_MAJORITY_TIE=NUM_MAJORITY_VOTE, THRESH=THRESHOLD, NUM_EXAMPLES=len(X_test))
                print("len y_pred_ens = ", len(y_pred_ens))
                print(y_pred_ens[0:10])

                print("Unique y_pred_ens: ", np.unique(y_pred_ens, return_counts=True))

                model_name_to_save = "ENS_" + MODEL_NAME + "_" + str(NUM_ENS) + "_models_" + str(synth_type) + "Synth_" + "th_" + str(THRESHOLD) + "_Major_" + str(NUM_MAJORITY_VOTE)
                print(f"Model_name_to_save = {model_name_to_save}")
                

                # ## Append to dataframe and csv file.
                df_models = appendAndReturn_DataframeRes(df_models=df_models, clf_model_name=model_name_to_save,
                                                Y_true=df_test["label"].values, Y_predicted=y_pred_ens)
                # ## Delete lists.
                del count_zeros
                del count_ones
                del y_pred_ens
        
        del y_preds_all


Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-NoSynth-7Sept-4am-0.h5 done !!
Predicting for Model_iter = 0
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-NoSynth-7Sept-4am-1.h5 done !!
Predicting for Model_iter = 1
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-NoSynth-7Sept-4am-2.h5 done !!
Predicting for Model_iter = 2
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-NoSynth-7Sept-4am-3.h5 done !!
Predicting for Model_iter = 3
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-NoSynth-7Sept-4am-4.h5 done !!
Predicting for Model_iter = 4
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-NoSynth-7Sept-4am-5.h5 done !!
Predicting for Model_iter = 5
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-num

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in long_scalars
  
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


len y_pred_ens =  5033
[1 1 1 1 1 1 1 1 1 1]
Unique y_pred_ens:  (array([0, 1]), array([ 124, 4909]))
Model_name_to_save = ENS_LSTM_10_models_NoSynth_th_0.5_Major_9
NUM_ENS = 10, THRESHOLD = 0.5, NUM_MAJORITY_VOTE = 8
len y_pred_ens =  5033
[1 1 1 1 1 1 1 1 1 1]
Unique y_pred_ens:  (array([0, 1]), array([ 139, 4894]))
Model_name_to_save = ENS_LSTM_10_models_NoSynth_th_0.5_Major_8
NUM_ENS = 10, THRESHOLD = 0.5, NUM_MAJORITY_VOTE = 6
len y_pred_ens =  5033
[1 1 1 1 1 1 1 1 1 1]
Unique y_pred_ens:  (array([0, 1]), array([ 181, 4852]))
Model_name_to_save = ENS_LSTM_10_models_NoSynth_th_0.5_Major_6
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-WordSynth-7Sept-4am-0.h5 done !!
Predicting for Model_iter = 0
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-10-WordSynth-7Sept-4am-1.h5 done !!
Predicting for Model_iter = 1
Loading model from file /content/drive/My Drive/Ensembling/Ens-Models/ens-LSTM-numModels-

In [13]:
def calculate_f1(p, r, EPSILON=0.0000001):
    return (2*p*r)/(p+r+EPSILON)

print(calculate_f1(0.81, 0.84))

df_models["Minority-F1(%)"] = df_models.apply(lambda row : calculate_f1(row['TNR/Specificity(%)'], row['Negative Pred. Value (%)']), axis=1)
print(df_models.columns.values)

0.8247272227438047
['Model Name' 'Confusion Matrix' '[TN, FP, FN, TP]' 'TNR/Specificity(%)'
 'FPR(%)' 'TPR/Recall(%)' 'FNR(%)' 'F1-Score(%)' 'Precision(%)'
 'Negative Pred. Value (%)' 'Accuracy(%)' 'CLF-Report-SKL'
 'Minority-F1(%)']


In [14]:
final_df = df_models.sort_values(by=['Minority-F1(%)'], ascending=False)
subset_df = final_df[['Model Name', 'Confusion Matrix', 'Negative Pred. Value (%)', 'TNR/Specificity(%)', 'Minority-F1(%)', 'CLF-Report-SKL']]

In [15]:
print(f"len subset_df = {len(subset_df)}")

len subset_df = 96


In [16]:
display(subset_df)

,Model Name,Confusion Matrix,Negative Pred. Value (%),TNR/Specificity(%),Minority-F1(%),CLF-Report-SKL
30,ENS_LSTM_10_models_CharSynth_th_0.35_Major_8,"[[132, 33], [22, 4846]]",85.714286,80.000000,82.758621,precision recall f1-score ...
25,ENS_LSTM_10_models_CharSynth_th_0.1_Major_9,"[[127, 38], [16, 4852]]",88.811189,76.969697,82.467532,precision recall f1-score ...
34,ENS_LSTM_10_models_CharSynth_th_0.5_Major_8,"[[131, 34], [22, 4846]]",85.620915,79.393939,82.389937,precision recall f1-score ...
26,ENS_LSTM_10_models_CharSynth_th_0.1_Major_8,"[[134, 31], [27, 4841]]",83.229814,81.212121,82.208589,precision recall f1-score ...
72,ENS_LSTM_5_models_CharSynth_th_0.1_Major_5,"[[122, 43], [10, 4858]]",92.424242,73.939394,82.154882,precision recall f1-score ...
...,...,...,...,...,...,...
19,ENS_LSTM_10_models_WordSynth_th_0.35_Major_6,"[[142, 23], [84, 4784]]",62.831858,86.060606,72.634271,precision recall f1-score ...
75,ENS_LSTM_5_models_CharSynth_th_0.1_Major_-1,"[[135, 30], [74, 4794]]",64.593301,81.818182,72.192513,precision recall f1-score ...
74,ENS_LSTM_5_models_CharSynth_th_0.1_Major_3,"[[135, 30], [74, 4794]]",64.593301,81.818182,72.192513,precision recall f1-score ...
15,ENS_LSTM_10_models_WordSynth_th_0.1_Major_6,"[[142, 23], [90, 4778]]",61.206897,86.060606,71.536524,precision recall f1-score ...


In [17]:
def get_synthesis_type(name_model):
    print(f"name_model = {name_model}")
    if "CharSyn" in name_model:
        return "Char"
    elif "WordSyn" in name_model:
        return "Word"
    elif "NoSyn" in name_model:
        return "No"
    elif "SW" in name_model:
        return "SW"
    else:
        return "UNKNOWN"
print(get_synthesis_type("FOC-CNN-WordSyn-th-0.35-NormalWts"))

## Call this function
new_df = subset_df
new_df['SynthesisType'] = new_df.apply(lambda row : get_synthesis_type(row['Model Name']), axis=1)

## Group by this type of synthesis
gkk = new_df.groupby(['SynthesisType']) 

name_model = FOC-CNN-WordSyn-th-0.35-NormalWts
Word
name_model = ENS_LSTM_10_models_CharSynth_th_0.35_Major_8
name_model = ENS_LSTM_10_models_CharSynth_th_0.1_Major_9
name_model = ENS_LSTM_10_models_CharSynth_th_0.5_Major_8
name_model = ENS_LSTM_10_models_CharSynth_th_0.1_Major_8
name_model = ENS_LSTM_5_models_CharSynth_th_0.1_Major_5
name_model = ENS_LSTM_10_models_WordSynth_th_0.35_Major_9
name_model = ENS_LSTM_10_models_SWSynth_th_0.5_Major_9
name_model = ENS_LSTM_10_models_SWSynth_th_0.35_Major_9
name_model = ENS_LSTM_10_models_SWSynth_th_0.1_Major_9
name_model = ENS_LSTM_10_models_WordSynth_th_0.35_Major_10
name_model = ENS_LSTM_10_models_WordSynth_th_0.1_Major_10
name_model = ENS_LSTM_5_models_SWSynth_th_0.35_Major_4
name_model = ENS_LSTM_5_models_SWSynth_th_0.5_Major_4
name_model = ENS_LSTM_10_models_WordSynth_th_0.1_Major_9
name_model = ENS_LSTM_10_models_WordSynth_th_0.5_Major_10
name_model = ENS_LSTM_10_models_CharSynth_th_0.35_Major_9
name_model = ENS_LSTM_10_models_WordSynt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
csv_file = "/content/drive/My Drive/CSV-" + MODEL_NAME + "-Ens-All-8Sept.csv"
itr = 0
for g in gkk:
    print(type(g[0]), type(g[1]))
    df = g[1]
    # csv_file = "/content/drive/My Drive/Focal-Loss-10-Epochs-Try-7Sept/Notebooks/CSV-Focal-10ep-" + g[0] + "-Synth" "-7Sept.csv"
    df_to_save = df.sort_values(by=['Minority-F1(%)'], ascending=False)
    if itr == 0:
        df_to_save.to_csv(csv_file, index=False, mode='w')
    else:
        df_to_save.to_csv(csv_file, index=False, mode='a', header=False)
    itr += 1

    print(f"Saved to {csv_file}")
    del df
    del df_to_save

<class 'str'> <class 'pandas.core.frame.DataFrame'>
Saved to /content/drive/My Drive/CSV-LSTM-Ens-All-8Sept.csv
<class 'str'> <class 'pandas.core.frame.DataFrame'>
Saved to /content/drive/My Drive/CSV-LSTM-Ens-All-8Sept.csv
<class 'str'> <class 'pandas.core.frame.DataFrame'>
Saved to /content/drive/My Drive/CSV-LSTM-Ens-All-8Sept.csv
<class 'str'> <class 'pandas.core.frame.DataFrame'>
Saved to /content/drive/My Drive/CSV-LSTM-Ens-All-8Sept.csv


In [19]:
# # csv_file = "/content/drive/My Drive/CNN-Ensembling-All-Models-7Sept.csv"
# subset_df.to_csv(csv_file, index=False)
# print(f"Saved to file {csv_file} done !")

In [20]:
display(subset_df) ## For 10 epochs.

,Model Name,Confusion Matrix,Negative Pred. Value (%),TNR/Specificity(%),Minority-F1(%),CLF-Report-SKL,SynthesisType
30,ENS_LSTM_10_models_CharSynth_th_0.35_Major_8,"[[132, 33], [22, 4846]]",85.714286,80.000000,82.758621,precision recall f1-score ...,Char
25,ENS_LSTM_10_models_CharSynth_th_0.1_Major_9,"[[127, 38], [16, 4852]]",88.811189,76.969697,82.467532,precision recall f1-score ...,Char
34,ENS_LSTM_10_models_CharSynth_th_0.5_Major_8,"[[131, 34], [22, 4846]]",85.620915,79.393939,82.389937,precision recall f1-score ...,Char
26,ENS_LSTM_10_models_CharSynth_th_0.1_Major_8,"[[134, 31], [27, 4841]]",83.229814,81.212121,82.208589,precision recall f1-score ...,Char
72,ENS_LSTM_5_models_CharSynth_th_0.1_Major_5,"[[122, 43], [10, 4858]]",92.424242,73.939394,82.154882,precision recall f1-score ...,Char
...,...,...,...,...,...,...,...
19,ENS_LSTM_10_models_WordSynth_th_0.35_Major_6,"[[142, 23], [84, 4784]]",62.831858,86.060606,72.634271,precision recall f1-score ...,Word
75,ENS_LSTM_5_models_CharSynth_th_0.1_Major_-1,"[[135, 30], [74, 4794]]",64.593301,81.818182,72.192513,precision recall f1-score ...,Char
74,ENS_LSTM_5_models_CharSynth_th_0.1_Major_3,"[[135, 30], [74, 4794]]",64.593301,81.818182,72.192513,precision recall f1-score ...,Char
15,ENS_LSTM_10_models_WordSynth_th_0.1_Major_6,"[[142, 23], [90, 4778]]",61.206897,86.060606,71.536524,precision recall f1-score ...,Word
